In [1]:
import cx_Oracle
import os
import pandas as pd
import xlwings as xw
import json
import datetime as dt
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()


In [76]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

### POD xl sheet 만들기

In [29]:
def bring_pod_sht_data():
    soout_col_names = ['SO_INDEX', 'SHT_ROW_IDX', 'PERSON_IN_CHARGE', 'SHIP_DATE', 'DM_KEY', 'POD_KEY', 'IS_LOCAL','POD_DATE', 'TIMELINE']
    sel_sth = wb_cy.selection.sheet
    idx_cel =  sel_sth.range("C2")
    idx_cel.api.Validation.Delete()
    df_soout = pd.DataFrame(cursor.execute("select * from so_out where pod_date is null").fetchall())
    df_soout.columns = soout_col_names
    df_soout = df_soout.sort_values('SO_INDEX')
    df_soout.set_index('SO_INDEX',inplace=True)
    df_soout['SO_DETAIL'] = None
    df_soout['LC_DETAIL'] = None


    for so_index in range(len(df_soout)):
        tmp_si_idx = df_soout['SHT_ROW_IDX'].iloc[so_index]
        tmp_lc_idx = df_soout['IS_LOCAL'].iloc[so_index]
        si_and_lc_list = [get_each_index_num(tmp_si_idx), get_each_index_num(tmp_lc_idx)]
        df_so = pd.DataFrame()
        df_lc = pd.DataFrame()
        for i,val in enumerate(si_and_lc_list):

            if val == 'local':
                None
            else:
                if type(val) is list:
                    idx_list = val
                elif type(val) is dict:
                    idx_list=val['idx_list']
                else:
                    idx_list = [val]
                for idx in idx_list:
                    if i == 0:
                        db_obj = cursor.execute(f"SELECT * FROM SHIPMENT_INFORMATION WHERE SI_INDEX = '{idx}'").fetchone()
                        df_db_obj = pd.DataFrame([db_obj])
                        df_so = pd.concat([df_so,df_db_obj])
                    else:
                        db_obj = cursor.execute(f"SELECT * FROM LOCAL_LIST WHERE LC_INDEX = '{idx}'").fetchone()
                        df_db_obj = pd.DataFrame([db_obj])
                        df_lc = pd.concat([df_lc,df_db_obj])
                    if not df_so.empty:
                        df_soout['SO_DETAIL'].iloc[so_index] = df_so
                    if not df_lc.empty:
                        df_soout['LC_DETAIL'].iloc[so_index] = df_lc

    df_soout['ORDER_NO'] = None
    df_soout['SHIP_TO'] = None
    for so_index in range(len(df_soout)):
        order_list = []
        shipto_list = []
        so_detail = df_soout['SO_DETAIL'].iloc[so_index]
        lc_detail = df_soout['LC_DETAIL'].iloc[so_index]
        if type(so_detail) != type(None):
            order_list = order_list + list(set(df_soout['SO_DETAIL'].iloc[so_index][6]))
            shipto_list = shipto_list + list(set(df_soout['SO_DETAIL'].iloc[so_index][9]))
        if type(lc_detail) != type(None):
            order_list = order_list + list(set(df_soout['LC_DETAIL'].iloc[so_index][5]))
            shipto_list = shipto_list + list(set(df_soout['LC_DETAIL'].iloc[so_index][8]))

        none_count = order_list.count(None)
        if none_count>0 :
            for i in range(none_count):
                none_idx = order_list.index(None)
                order_list.pop(none_idx)

        none_count_lc = shipto_list.count(None)
        if none_count_lc>0 :
            for i in range(none_count_lc):
                none_idx = shipto_list.index(None)
                shipto_list.pop(none_idx)    
        order_num = ", ".join(order_list)
        ship_to = ", ".join(shipto_list)
        df_soout['ORDER_NO'].iloc[so_index] = order_num
        df_soout['SHIP_TO'].iloc[so_index] = ship_to

    db_del_med = pd.DataFrame(cursor.execute("select * from DELIVERY_METHOD").fetchall())
    db_pod_med = pd.DataFrame(cursor.execute("select * from POD_METHOD").fetchall())
    for i in range(len(db_del_med)):
        df_soout['DM_KEY'].replace(db_del_med[0][i],db_del_med[1][i],inplace=True)
    for j in range(len(db_pod_med)):
        df_soout['POD_KEY'].replace(db_pod_med[0][j],db_pod_med[1][j],inplace=True)

    df_soout['TIMELINE'] = df_soout['TIMELINE'].map(lambda e: json.loads(e)['data'][-1]['c'])

    is_lc_list = df_soout[df_soout['IS_LOCAL'].str.contains('lc_')]['IS_LOCAL'].index
    for lc_idx in is_lc_list:
        df_soout['IS_LOCAL'].loc[lc_idx] = 'is_local'


    df_soout= df_soout[['POD_KEY','SHIP_DATE','ORDER_NO', 'SHIP_TO','PERSON_IN_CHARGE','IS_LOCAL', 'POD_DATE',  'DM_KEY',
            'TIMELINE']]

    sel_sth.range("A9").value = df_soout
    
    

In [71]:

idx_cel =  sel_sth.range("C2")
idx_cel.api.Validation.Delete()
last_row = sel_sth.range('A1048576').end('up').row
if last_row < 10 :
    last_row = 10
so_idx_list = sel_sth.range("A10:A"+str(last_row)).options(numbers=lambda x: str(int(x))).value
idx_list = ",".join(so_idx_list)
idx_cel.api.Validation.Add(Type=3, Formula1=idx_list)

In [61]:
def see_details():
    
    sel_sth = wb_cy.selection.sheet
    wb_pf = xw.Book('print_form.xlsx')
    pod_sht_pf =wb_pf.sheets['POD_DETAIL']
    
    soout_col_names = ['SO_INDEX', 'SHT_ROW_IDX', 'PERSON_IN_CHARGE', 'SHIP_DATE', 'DM_KEY', 'POD_KEY', 'IS_LOCAL','POD_DATE', 'TIMELINE']
    call_index =  sel_sth.range("C2").options(numbers=int).value
    df_soout = pd.DataFrame(cursor.execute("select * from so_out where pod_date is null").fetchall())
    df_soout.columns = soout_col_names
    df_soout = df_soout.sort_values('SO_INDEX')
    df_soout.set_index('SO_INDEX',inplace=True)
    df_soout['SO_DETAIL'] = None
    df_soout['LC_DETAIL'] = None


    for so_index in range(len(df_soout)):
        tmp_si_idx = df_soout['SHT_ROW_IDX'].iloc[so_index]
        tmp_lc_idx = df_soout['IS_LOCAL'].iloc[so_index]
        si_and_lc_list = [get_each_index_num(tmp_si_idx), get_each_index_num(tmp_lc_idx)]
        df_so = pd.DataFrame()
        df_lc = pd.DataFrame()
        for i,val in enumerate(si_and_lc_list):

            if val == 'local':
                None
            else:
                if type(val) is list:
                    idx_list = val
                elif type(val) is dict:
                    idx_list=val['idx_list']
                else:
                    idx_list = [val]
                for idx in idx_list:
                    if i == 0:
                        db_obj = cursor.execute(f"SELECT * FROM SHIPMENT_INFORMATION WHERE SI_INDEX = '{idx}'").fetchone()
                        df_db_obj = pd.DataFrame([db_obj])
                        df_so = pd.concat([df_so,df_db_obj])
                    else:
                        db_obj = cursor.execute(f"SELECT * FROM LOCAL_LIST WHERE LC_INDEX = '{idx}'").fetchone()
                        df_db_obj = pd.DataFrame([db_obj])
                        df_lc = pd.concat([df_lc,df_db_obj])
                    if not df_so.empty:
                        df_soout['SO_DETAIL'].iloc[so_index] = df_so
                    if not df_lc.empty:
                        df_soout['LC_DETAIL'].iloc[so_index] = df_lc
    df_details = df_soout[['SO_DETAIL','LC_DETAIL']].loc[int(call_index)]
    
    so_col_name= list(pd.DataFrame(cursor.execute("select * from cols where table_name = 'SHIPMENT_INFORMATION'").fetchall())[1])
    lc_col_name= list(pd.DataFrame(cursor.execute("select * from cols where table_name = 'LOCAL_LIST'").fetchall())[1])

    last_row = pod_sht_pf.range("A1048576").end('up').row
    pod_sht_pf.range((1,"A"),(last_row,"Z")).clear_contents()

    try : 
        so_detail = df_details['SO_DETAIL']
        so_detail.columns = so_col_name
        so_detail.set_index('SI_INDEX',inplace=True)
        pod_sht_pf.range('A1').value = so_detail
    except:
        None

    last_row = pod_sht_pf.range("A1048576").end('up').row + 1

    try :    
        lc_detail = df_details['LC_DETAIL']
        lc_detail.columns = lc_col_name
        lc_detail.set_index('LC_INDEX',inplace=True)
        pod_sht_pf.range('A'+str(last_row)).value = lc_detail
    except:
        None
    pod_sht_pf.range("A1").select()

In [28]:
wb_pf = xw.Book('print_form.xlsx')
pod_sht_pf =wb_pf.sheets['POD_DETAIL']
last_row = pod_sht_pf.range("A1048576").end('up').row
pod_sht_pf.range((1,"A"),(last_row,"Z")).clear_contents()

try : 
    so_detail = df_details['SO_DETAIL'].loc[1]
    so_detail.columns = so_col_name
    so_detail.set_index('SI_INDEX',inplace=True)
    pod_sht_pf.range('A1').value = so_detail
except:
    None
    
last_row = pod_sht_pf.range("A1048576").end('up').row + 1

try :    
    lc_detail = df_details['LC_DETAIL'].loc[1]
    lc_detail.columns = lc_col_name
    lc_detail.set_index('LC_INDEX',inplace=True)
    pod_sht_pf.range('A'+str(last_row)).value = lc_detail
except:
    None

# 


AttributeError: 'NoneType' object has no attribute 'columns'

In [75]:
## POD_DONE

answer = wb_cy.app.alert("POD DONE을 진행하시겠습니까?","POD Confirm",buttons='yes_no_cancel')
if answer != 'yes':
    wb_cy.app.alert("종료합니다.","Quit")
#     return
sel_sth = wb_cy.selection.sheet
so_idx_cell = sel_sth.range("C2").options(numbers=lambda x: str(int(x)))


'6'

In [90]:
df_so = pd.DataFrame([cursor.execute(f"select * from so_out where so_index = '{so_idx_cell.value}'").fetchone()])
df_so[6][0]
si_index = get_each_index_num(df_so[1][0])
lc_index = get_each_index_num(df_so[6][0])
si_lc_idx = [si_index,lc_index]
for i, val in enumerate(si_lc_idx):
    if val == 'local':
        None
    else:
        if type(val) is list:
            idx_list = val
        elif type(val) is dict:
            idx_list=val['idx_list']
        else:
            idx_list = [val]
        for idx in idx_list:
            if i == 0: #si
                
            else: #lc

20474


In [112]:
def pod_done():
    sel_sth = wb_cy.selection.sheet
    date_cell = sel_sth.range("C3").options(dates=my_date_handler)
    date_answer = wb_cy.app.alert(date_cell.value + " 의 날짜로 POD진행하시겠습니까?","POD Confirm",buttons='yes_no_cancel')
    if date_answer != 'yes':
        wb_cy.app.alert("DATE를 입력하시고 다시 진행해주세요. 종료합니다..","Quit")
    #     return

    answer = wb_cy.app.alert("POD DONE을 진행하시겠습니까?","POD Confirm",buttons='yes_no_cancel')
    if answer != 'yes':
        wb_cy.app.alert("종료합니다.","Quit")
    #     return

    call_index =  sel_sth.range("C2").options(numbers=lambda x: str(int(x))).value
    if call_index == None:
        wb_cy.app.alert("SO_INDEX를 DropDownList에서 하나 선택한 후 버튼을 눌러주세요. 종료합니다.",'Quit')
    #     return

    soout_col_names = ['SO_INDEX', 'SHT_ROW_IDX', 'PERSON_IN_CHARGE', 'SHIP_DATE', 'DM_KEY', 'POD_KEY', 'IS_LOCAL','POD_DATE', 'TIMELINE']

    df_soout = pd.DataFrame(cursor.execute(f"select * from so_out where so_index = '{call_index}'").fetchall())
    df_soout.columns = soout_col_names
    df_soout = df_soout.sort_values('SO_INDEX')
    df_soout.set_index('SO_INDEX',inplace=True)
    df_soout['SO_DETAIL'] = None
    df_soout['LC_DETAIL'] = None


    for so_index in range(len(df_soout)):
        tmp_si_idx = df_soout['SHT_ROW_IDX'].iloc[so_index]
        tmp_lc_idx = df_soout['IS_LOCAL'].iloc[so_index]
        si_and_lc_list = [get_each_index_num(tmp_si_idx), get_each_index_num(tmp_lc_idx)]
        df_so = pd.DataFrame()
        df_lc = pd.DataFrame()
        for i,val in enumerate(si_and_lc_list):

            if val == 'local':
                None
            else:
                if type(val) is list:
                    idx_list = val
                elif type(val) is dict:
                    idx_list=val['idx_list']
                else:
                    idx_list = [val]
                for idx in idx_list:
                    if i == 0:
                        db_obj = cursor.execute(f"SELECT * FROM SHIPMENT_INFORMATION WHERE SI_INDEX = '{idx}'").fetchone()
                        df_db_obj = pd.DataFrame([db_obj])
                        df_so = pd.concat([df_so,df_db_obj])
                    else:
                        db_obj = cursor.execute(f"SELECT * FROM LOCAL_LIST WHERE LC_INDEX = '{idx}'").fetchone()
                        df_db_obj = pd.DataFrame([db_obj])
                        df_lc = pd.concat([df_lc,df_db_obj])
                    if not df_so.empty:
                        df_soout['SO_DETAIL'].iloc[so_index] = df_so
                    if not df_lc.empty:
                        df_soout['LC_DETAIL'].iloc[so_index] = df_lc
    df_details = df_soout[['SO_DETAIL','LC_DETAIL']].loc[int(call_index)]
    try:
        so_detail = df_details['SO_DETAIL']
        ShipmentInformation.update_pod_date(so_detail,date_cell.value)

    except:
        None

    try:
        lc_detail = df_details['LC_DETAIL']
        ShipmentInformation.update_pod_date(lc_detail,date_cell.value)

    except:
        None    


In [120]:
list(df_details['SO_DETAIL'][0])

TypeError: 'NoneType' object is not subscriptable

In [113]:
wb_cy.sheets['Shipment information'].activate()

In [4]:
def get_each_index_num(get_idx_str):
    """
    DW테이블 SO_OUT상의 si_index 및 is_local 컬럼값을 넣으면 해당 row의 고유 키값을 dict형태로 반환

    example : get_idx_str='si_13384A14171A14243C14244'  ==>
    {'out_sht_id':'si','idx_list':[13384, 14171, 14243, 14244]}
    """
    del_sht_id = get_idx_str.split('_')[0]
    get_idx_str = get_idx_str.split('_')[1]
    count_A = get_idx_str.count("A")
    count_C = get_idx_str.count("C")
    if count_A == 0 and count_C == 0:
        return get_idx_str
    procs_1 = get_idx_str.split("A")
    procs_1
    A_list = []
    C_list = []
    for val in procs_1:
        if val.count("C") > 0 :
            C_list.append(val)
        else :
            A_list.append(int(val))
    
    for c_val in C_list:      
        tmp_c = c_val.split('C')
        tmp_diff = int(tmp_c[1])-int(tmp_c[0])
        fin_C_list = []
        for i, val in enumerate(range(tmp_diff+1)):
            fin_C_list.append(int(tmp_c[0]) + i)
        A_list = A_list+fin_C_list
            
    return {'out_sht_id':del_sht_id,'idx_list':A_list}